In [1]:
#
# Simple example of a Julia linear pogramming model
#

#

 
using LinearAlgebra

const MPS_EXAMPLE::String = """
NAME          EXAMPLE
ROWS
 N  COST
 L  C1
 L  C2
 L  C3
COLUMNS
    x1        COST       3
    x1        C1         1
    x1        C2         1
    x2        COST       2
    x2        C1         1
    x2        C3         1
RHS
    RHS       C1         4
    RHS       C2         2
    RHS       C3         3
BOUNDS
 UP BND       x1         2
 UP BND       x2         3
ENDATA
"""

"NAME          EXAMPLE\nROWS\n N  COST\n L  C1\n L  C2\n L  C3\nCOLUMNS\n    x1        COST       3\n    x1        C1         1\n    x1        C2         1\n    x2        COST       2\n    x2        C1         1\n    x2        C3         1\nRHS\n    RHS       C1         4\n    RHS       C2         2\n    RHS       C3         3\nBOUNDS\n UP BND       x1         2\n UP BND       x2         3\nENDATA\n"

For the example provided:

Objective function coefficients: 
$c = [3, 2]$

Constraints matrix: 
$ A = \left [ \begin{matrix} 
1 & 1 \\ 
1 & 0 \\ 
0 & 1 
\end{matrix} \right ] $

Right-hand side vector: 
$b = [4, 2, 3]$

In [2]:
function simplex(c, A, b)
    m, n = size(A) # Get the number of constraints (m) and the number of variables (n)
    
    # println(A)
    
    tableau = hcat(A, Matrix{Float64}(I, m, m), b) # Create the initial tableau by combining A, an identity matrix for slack variables, and the right-hand side vector b
    
    # println(tableau)
    
    tableau = vcat(tableau, hcat(c', zeros(1, m + 1))) # Add the objective function row to the tableau
    
    # println(tableau)
    
    while any(tableau[end, 1:end-1] .> 0) # While there are positive coefficients in the objective function row
        col = findmax(tableau[end, 1:end-1])[2] # Identify the entering variable (column with the most positive coefficient)
        if all(tableau[1:end-1, col] .<= 0) # Check if the problem is unbounded
            error("Unbounded solution") # If all entries in the column are non-positive, the solution is unbounded
        end
        ratios = tableau[1:end-1, end] ./ tableau[1:end-1, col] # Compute the ratios of the right-hand side to the pivot column
        ratios[ratios .<= 0] .= Inf # Ignore non-positive ratios by setting them to infinity
        row = argmin(ratios) # Identify the pivot row (the row with the minimum positive ratio)
        
        tableau[row, :] /= tableau[row, col] # Normalize the pivot row
        for i in 1:size(tableau, 1) # Perform row operations to zero out the other entries in the pivot column
            if i != row
                tableau[i, :] -= tableau[i, col] * tableau[row, :] # Update the row by subtracting the pivot row
            end
        end
    end
    
    solution = zeros(n) # Initialize the solution vector
    for j in 1:n # Extract the solution for the original variables
        column = tableau[1:end-1, j]
        if sum(column .== 1) == 1 && sum(column .== 0) == m - 1 # Check if the column corresponds to a basic variable
            row = findfirst(column .== 1) # Find the row index of the basic variable
            solution[j] = tableau[row, end] # Set the value of the basic variable in the solution
        end
    end
    
    return solution, tableau[end, end] # Return the optimal solution and the maximum value of the objective function
end


simplex (generic function with 1 method)

In [4]:
# Example problem

c = [3.0, 2.0]
A = [1.0 1.0; 1.0 0.0; 0.0 1.0]
b = [4.0, 2.0, 3.0]

execution_time = @elapsed solution, maximum_value = simplex(c, A, b)
println("\nExecution time: $execution_time seconds")

println("Optimal solution: ", solution)
println("Maximum value: ", maximum_value)


Execution time: 2.2708e-5 seconds
Optimal solution: [2.0, 2.0]
Maximum value: -10.0
